In [1]:
#extract all subdatasets from h5 and convert them to tif 
#make all these tifs into cogs
#stack all cogs and create a tif 
#oconvert final tif to COG

In [2]:
import gdal
import rasterio
import numpy as np
import os
import glob
from rasterio.mask import mask

In [3]:
ds = gdal.Open("Downloads/MYD08_D3.A2020289.061.2020291075457.hdf", gdal.GA_ReadOnly)
ds.GetSubDatasets()

[('HDF4_EOS:EOS_GRID:"Downloads/MYD08_D3.A2020289.061.2020291075457.hdf":mod08:Solar_Zenith_Mean',
  '[180x360] Solar_Zenith_Mean mod08 (16-bit integer)'),
 ('HDF4_EOS:EOS_GRID:"Downloads/MYD08_D3.A2020289.061.2020291075457.hdf":mod08:Solar_Zenith_Standard_Deviation',
  '[180x360] Solar_Zenith_Standard_Deviation mod08 (16-bit integer)'),
 ('HDF4_EOS:EOS_GRID:"Downloads/MYD08_D3.A2020289.061.2020291075457.hdf":mod08:Solar_Zenith_Minimum',
  '[180x360] Solar_Zenith_Minimum mod08 (16-bit integer)'),
 ('HDF4_EOS:EOS_GRID:"Downloads/MYD08_D3.A2020289.061.2020291075457.hdf":mod08:Solar_Zenith_Maximum',
  '[180x360] Solar_Zenith_Maximum mod08 (16-bit integer)'),
 ('HDF4_EOS:EOS_GRID:"Downloads/MYD08_D3.A2020289.061.2020291075457.hdf":mod08:Solar_Zenith_Pixel_Counts',
  '[180x360] Solar_Zenith_Pixel_Counts mod08 (16-bit integer)'),
 ('HDF4_EOS:EOS_GRID:"Downloads/MYD08_D3.A2020289.061.2020291075457.hdf":mod08:Solar_Azimuth_Mean',
  '[180x360] Solar_Azimuth_Mean mod08 (16-bit integer)'),
 ('HDF

In [4]:
#temp mean, min, max corr to 887, 889, 890
#ozone mean, min, max corr to 839, 841, 842
#cloud frac mean, min, max corr to 253, 255, 256
#water vap mean, min, max corr to 863, 865, 866 

In [6]:
os.chdir("/Users/manyachadha/weather")
os.getcwd()

'/Users/manyachadha/weather'

In [7]:
def tif_convert(rlayer):
    EPSG = '-a_srs EPSG:4326 -a_nodata -32768 -co "COMPRESS=PACKBITS"'
    translateOptionText = EPSG+" -a_ullr " + "-180" + " " + "90" + " " + "180" + " " + "-90"
    translateoptions = gdal.TranslateOptions(gdal.ParseCommandLine(translateOptionText))
    gdal.Translate(f'{opfilename}.tif',rlayer, options=translateoptions)

In [12]:
#creating tifs of all subdatasets
for i in [887, 889, 890, 839, 841, 842, 253,255,256, 863,865,866]:
    
    subhdflayer = ds.GetSubDatasets()[i][0]
    rlayer = gdal.Open(subhdflayer, gdal.GA_ReadOnly)
    opfilename = subhdflayer.split(':')[4]
    
    tif_convert(rlayer)

In [13]:
#converting the tifs into cogs 
tifs = glob.glob("/Users/manyachadha/weather/*.tif")
for tif in tifs:
    name = tif.split('/')[4].split('.')[0]
    root = '/Users/manyachadha/weather'
    os.system("gdal_translate {}/{}.tif {}/{}_stack_COG.tif -co COMPRESS=LZW -co TILED=YES -co BLOCKXSIZE=512 -co BLOCKYSIZE=512 -co COMPRESS=LZW -co PREDICTOR=2 -co COPY_SRC_OVERVIEWS=YES -co BIGTIFF=YES".format(root,name,root,name))
    

In [18]:
#stacking the cogs to create a single tif
root = "/Users/manyachadha/weather"
cogs = glob.glob("/Users/manyachadha/weather/*COG.tif")
for cog in cogs:
    name = cog.split("/")[4].split(".")[0]
#     print(name)
    with rasterio.open(cogs[0]) as src0:
                meta = src0.meta
    meta.update(count = len(cogs))
    with rasterio.open(root + "/stacked" + '/stack.tif', 'w', **meta) as dst:
        for id, layer in enumerate(cogs, start=1):
            with rasterio.open(layer) as src1:
                        dst.write_band(id, src1.read(1))

In [21]:
#convering the final stacked tif into a cog
root = "/Users/manyachadha/weather"
os.system("gdal_translate {}/stacked/stack.tif {}/stacked/stack_COG.tif -co COMPRESS=LZW -co TILED=YES -co BLOCKXSIZE=512 -co BLOCKYSIZE=512 -co COMPRESS=LZW -co PREDICTOR=2 -co COPY_SRC_OVERVIEWS=YES -co BIGTIFF=YES".format(root,root))


0

In [46]:
for i in glob.glob("/Users/manyachadha/weather/*"):
    if i.endswith(".xml"):
        os.remove(i)